In [2]:
import numpy as np
import tqdm
embedding_path='raw_data/rotten_tomatoes/embeddings/glove.6B.200d.txt'
lines = []
with open(embedding_path, encoding='utf8') as file:
    lines = file.readlines()
    file.close()
embedding_tensor = []
token2id = {}
token2id['<unk>'] = 0
token2id['<pad>'] = 1
for id, l in tqdm.tqdm(enumerate(lines)):
    word, emb = l.split()[0], l.split()[1:]
    vector = [float(x) for x in emb ]
    if id == 0:
        embedding_tensor.append( np.zeros( len(vector) ) )
        embedding_tensor.append( np.zeros( len(vector) ) )
    embedding_tensor.append(vector)
    token2id[word] = id+2
embedding_tensor = np.array(embedding_tensor, dtype=np.float32)

400000it [00:20, 19323.76it/s]


In [36]:
import pandas as pd
from utils.embedding import get_indices_tensor
from utils.contractions import CONTRACTIONS_DICT
import re
def clean_punctuation(text):
    text = re.sub("""([!"#$%&'()*+, -./:;<=>?@[\]^_`{|}~])""", r' \1 ', text)
    text = re.sub('\s{2,}', ' ', text)
    return text
def expand_contractions(text):
    contractions_re=re.compile('(%s)' % '|'.join(CONTRACTIONS_DICT.keys()))
    def replace(match):
        return CONTRACTIONS_DICT[match.group(0)]
    return contractions_re.sub(replace, text)
def clean_spaces(text):
    text = " ".join(text.split())
    return text
root_path = 'raw_data/rotten_tomatoes/data/audience_reviews.csv'
df = pd.read_csv(root_path).dropna()
df['text']= df['Review'].apply(lambda x: x.lower())
df['text']=df['text'].apply(lambda x:expand_contractions(x))
df['text'] = df['text'].apply(lambda x:clean_spaces(x))
df['text']= df['text'].apply(lambda x:clean_punctuation(x))
df['text_list']= df['text'].apply(lambda x: x.split())
df['y'] = df['Rating'].apply(lambda x: x/5)
df['id'] = range(0, len(df))
df['x'] = df['text_list'].apply(lambda x: get_indices_tensor(x, token2id, 250))

In [39]:
df.reset_index(drop=True, inplace=True)

In [40]:
df.head()

,Show,Rating,Review,text,text_list,y,id,x
0,Loot,2.5,"I love the stars, but the show is riddled with...","i love the stars , but the show is riddled wit...","[i, love, the, stars, ,, but, the, show, is, r...",0.5,1,"[[tensor(43), tensor(837), tensor(2), tensor(1..."
1,Loot,4.5,Great show to watch. It's something that's eas...,great show to watch . it is something that is ...,"[great, show, to, watch, ., it, is, something,...",0.9,2,"[[tensor(355), tensor(275), tensor(6), tensor(..."
2,Loot,0.5,Another Apple tanker crashes into the reef wit...,another apple tanker crashes into the reef wit...,"[another, apple, tanker, crashes, into, the, r...",0.1,3,"[[tensor(172), tensor(3294), tensor(8406), ten..."
3,Loot,4.5,"This is such a fun, endearing show. It has a ...","this is such a fun , endearing show . it has a...","[this, is, such, a, fun, ,, endearing, show, ....",0.9,4,"[[tensor(39), tensor(16), tensor(127), tensor(..."
4,Loot,0.5,Terrible show - didn't laugh once. Acting is s...,terrible show - did not laugh once . acting is...,"[terrible, show, -, did, not, laugh, once, ., ...",0.1,5,"[[tensor(5512), tensor(275), tensor(13), tenso..."


In [253]:
j = 183#manually change this

In [ ]:
text_list = df['text_list'][j]
rating = df['Rating'][j]
print(rating)
text_df = {"text": text_list, "idx": np.arange(len(text_list))}
text_df = pd.DataFrame(text_df)
print(text_df.to_string())

In [ ]:
#manually add to sheet j, list of list of sentence indices start and end
#e.g. 0, [[0, 23], [52, 58]]
#choose which sentences make sense (1/4-3/4 of the review)
#skip any index that's too short or doesn't have punctuation to mark sentences
#get a mix of positive mixed negative reviews

In [255]:
import pickle
import json

In [ ]:
results = pickle.load(open('review_data/rotten_tomatoes/rationale_args_dict.pkl','rb'))
rationales = results['test_stats']['rationales']

In [ ]:
true_positive_running_count = 0
false_positive_running_count = 0
annotations = pd.read_csv('raw_data/rotten_tomatoes/annotations/annotations.csv')
for i, rationale in enumerate(rationales):
    rationale_list = rationale.split(" ")
    rationale_ids = [token_id for token_id, token in enumerate(rationale_list) if token != '_']
    annotation_ids = []
    for annotation in annotations.iloc[[i]]['annotations']:
        annotation_ids += list(range(annotation[0], annotation[1]))
    for rationale_id in rationale_ids:
        if rationale_id in annotation_ids:
            true_positive_running_count += 1
        else:
            false_positive_running_count += 1
precision = true_positive_running_count/(true_positive_running_count + false_positive_running_count)
print(precision)